In [ ]:
"""
Gaussian-Bernoulli RBM for ART IMAGES 

 Art images given in npy arrays

VanGogh64x64.npy
StillLife64x64.npy
Modigliani_paintings.npy
Abstractblue.npy

#image values are already normalized in [0 1]

libraries: 

!pip install opencv-python tensorflow keras numpy scikit-learn

"""


In [ ]:

"""
load data set and plot a few samples

available images

VanGogh64x64.npy
Monet64x64.npy
StillLife64x64.npy
Modigliani_paintings.npy
AbstractPaintings_Dataset.npy
Abstractblue.npy

"""

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

#data already normalized by 255, in the range [0 1]

#dataset shape: (num_img, size, size, channels)
data = np.load("Abstractblue.npy")

num_samples, img_size, canc, channels = data.shape

fig, axes = plt.subplots(5, 5, figsize=(8, 8))
for i, ax in enumerate(axes.flat):
    img = data[i]
    img = (img - img.min()) / (img.max() - img.min())  # Normalize for visualization
    ax.imshow(img)
    ax.axis("off")
    plt.suptitle("Data Images")

plt.show()

data.shape

print("data loaded")


In [ ]:
"""
RBM needs as input: 

numpy array filename = data (num_images x size x size x channels) 

number of hidden neurons (hidden_size) line 161 of the next cell 
number of epochs (epochs)  line 186 of the next cell 
learning rate  (hidden_size)
batch_size (batch_size)

"""


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import MinMaxScaler
from scipy.special import expit



# Define stable sigmoid function
#def sigmoid(x):
#    return 1 / (1 + np.exp(-x))
def sigmoid(x):
    return expit(x)

def batch_normalize(v):
    mean = np.mean(v, axis=0, keepdims=True)
    std = np.std(v, axis=0, keepdims=True) + 1e-8  # Avoid division by zero
    return (v - mean) / std
    
    
# Initialize RBM parameters
class GaussianBernoulliRBM:
    def __init__(self, visible_units, hidden_units, learning_rate, sig):
        self.v_units = visible_units
        self.h_units = hidden_units
        self.lr = learning_rate
        self.sigma = sig
        
        # Initialize weights and biases
        self.W = np.random.randn(self.v_units, self.h_units) * 0.01  # Small random values
        self.b_v = np.zeros(self.v_units)  # Visible biases
        self.b_h = np.zeros(self.h_units)  # Hidden biases

    # Forward pass: Sample hidden states given visible states
    def sample_hidden(self, v):
        h_mean = sigmoid( np.dot(v, self.W)/( (self.sigma) **2 ) + self.b_h )
        #h_sample = np.random.binomial(1, h_mean)  # Bernoulli sample
        h_sample = (np.random.uniform(size=h_mean.shape) < h_mean).astype(np.float32)
        return h_mean, h_sample

    # Backward pass: Sample visible states given hidden states
    def sample_visible(self, h):
        v_mean = np.dot(h, self.W.T) + self.b_v  # Gaussian visible layer
        v_sample = v_mean + self.sigma * np.random.randn(*v_mean.shape)  # Add Gaussian noise
        return v_mean, v_sample

    # Train RBM with Contrastive Divergence (CD-1)
    def train(self, data, epochs, batch_size):
        num_samples = data.shape[0]
        error_list =[]
        for epoch in range(epochs):
            np.random.shuffle(data)  # Shuffle data each epoch
            error = 0
            for i in range(0, num_samples, batch_size):
                v_0 = batch_normalize( data[i:i+batch_size] )  # Mini-batch

                # Positive phase
                h_0_mean, h_0_sample = self.sample_hidden(v_0)

                # Negative phase (Reconstruct visible layer)
                v_k_mean, v_k_sample = self.sample_visible(h_0_sample)
                h_k_mean, _ = self.sample_hidden(v_k_sample)

                # Update weights and biases
                self.W += self.lr * (np.dot(v_0.T, h_0_mean) - np.dot(v_k_sample.T, h_k_mean)) / batch_size
                self.b_v += self.lr * np.mean(v_0 - v_k_sample, axis=0)
                self.b_h += self.lr * np.mean(h_0_mean - h_k_mean, axis=0)

                error += np.mean( np.mean( np.abs( v_0 - v_k_mean ), axis=0 ) )

            print(f"\rEpoch {epoch+1}/{epochs} completed, error={error}", end=" ")

        error_list.append(error)
        return error_list


        # Train RBM with Contrastive Divergence (CD-1) + save hidden variables
    def train2(self, data, epochs, batch_size):
        num_samples = data.shape[0]
        error_list = []
        hidden_vars_list = []  # Store hidden variables for all images
    
        for epoch in range(epochs):
            np.random.shuffle(data)  # Shuffle data each epoch
            error = 0
            hidden_epoch = []  # Store hidden variables for the current epoch
    
            for i in range(0, num_samples, batch_size):
                v_0 = data[i:i+batch_size]  # Mini-batch
    
                # Positive phase
                h_0_mean, h_0_sample = self.sample_hidden(v_0)
    
                # Store hidden representations
                hidden_epoch.append(h_0_mean)
    
                # Negative phase (Reconstruct visible layer)
                v_k_mean, v_k_sample = self.sample_visible(h_0_sample)
                h_k_mean, _ = self.sample_hidden(v_k_sample)
    
                # Update weights and biases
                self.W += self.lr * (np.dot(v_0.T, h_0_mean) - np.dot(v_k_sample.T, h_k_mean)) / batch_size
                self.b_v += self.lr * np.mean(v_0 - v_k_sample, axis=0)
                self.b_h += self.lr * np.mean(h_0_mean - h_k_mean, axis=0)
    
                error += np.mean(np.mean(np.abs(v_0 - v_k_mean), axis=0))
    
            print(f"\rEpoch {epoch+1}/{epochs} completed, error={error}", end=" ")
    
            error_list.append(error)
            hidden_vars_list.append(np.vstack(hidden_epoch))  # Stack batch results
    
                # Select three dimensions to visualize
        n1, n2, n3 = 0, 1, 2  # Change these indices based on your preference
        hidden_vars_array = np.array(hidden_vars_list)
        # Extract the selected dimensions
        x = hidden_vars_array[:, n1]
        y = hidden_vars_array[:, n2]
        z = hidden_vars_array[:, n3]
        
        # 3D Plot
        fig = plt.figure(figsize=(12, 5))
        
        ax1 = fig.add_subplot(121, projection='3d')
        ax1.scatter(x, y, z, c='b', marker='o', alpha=0.6)
        ax1.set_xlabel(f'Hidden Variable {n1}')
        ax1.set_ylabel(f'Hidden Variable {n2}')
        ax1.set_zlabel(f'Hidden Variable {n3}')
        ax1.set_title('3D Scatter Plot of Hidden Variables')
        
        # 2D Plot (n1 vs. n2)
        ax2 = fig.add_subplot(122)
        ax2.scatter(x, y, c='r', alpha=0.6)
        ax2.set_xlabel(f'Hidden Variable {n1}')
        ax2.set_ylabel(f'Hidden Variable {n2}')
        ax2.set_title('2D Scatter Plot of Hidden Variables')
        
        plt.tight_layout()
        plt.show()
        return error_list, np.vstack(hidden_vars_list)  # Return hidden variables for all images
    
    
    # Generate samples from the trained RBM
    def generate_samples(self, num_samples, steps):
        v = np.random.randn(num_samples, self.v_units)  # Random initialization

        for _ in range(steps):  # Gibbs Sampling
            h_mean, h_sample = self.sample_hidden(v)
            v_mean, v = self.sample_visible(h_sample)

        return v_mean, h_mean # Return final generated visible samples and hidden variables 

"""
# main 
#data = np.load("VanGogh.npy")
#dataset shape:(855, 64, 64, 3)
#data already normalized by 255, in the range [0 1]
"""

num_samples, img_size, canc, channels = data.shape
hidden_size = 32  # number of hidden variables 

data_flat = data.reshape(num_samples, -1) #Array  num_samples x img_size^2

# Normalize images to [-1,1] for Gaussian-Bernoulli RBM
data_flat = (data_flat - 0.5) * 2  

fig, axes = plt.subplots(1, 8, figsize=(10, 2))
for i, ax in enumerate(axes):
    img = data[i].reshape(img_size, img_size, channels)
    img = (img - img.min()) / (img.max() - img.min())  # Normalize for visualization
    ax.imshow(img)
    ax.axis("off")
    plt.suptitle("Data Samples")

plt.savefig("DataSamples.png", format="png", dpi=300)  # Save as PNG
plt.savefig("DataSamples.eps", format="eps")  # Save as PDF
plt.savefig("DataSamples.pdf", format="pdf")  # Save as PDF
plt.savefig("DataSamples.tiff", format="tiff", dpi=300)  # Save as TIFF

plt.show()

print("start training")
# Train RBM on CBL Faces
rbm = GaussianBernoulliRBM(visible_units=channels*img_size**2, hidden_units=hidden_size, learning_rate=0.025, sig=0.1)
rbm.train2(data_flat, epochs=1000, batch_size=16)

# Generate and plot new face samples
generated_data, generated_hidden_var= rbm.generate_samples(num_samples=25, steps=100)

fig, axes = plt.subplots(5, 5, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    img = generated_data[i].reshape(img_size, img_size, channels)
    img = (img - img.min()) / (img.max() - img.min())  # Normalize for visualization
    ax.imshow(img, cmap='winter')
    ax.axis("off")
    plt.suptitle("Generated Samples")  

plt.savefig("GeneratedSamples.png", format="png", dpi=300)  # Save as PNG
plt.savefig("GeneratedSamples.eps", format="eps")  # Save as PDF
plt.savefig("GeneratedSamples.pdf", format="pdf")  # Save as PDF
plt.savefig("GeneratedSamples.tiff", format="tiff", dpi=300)  # Save as TIFF

plt.show()

hidden_n = int(np.sqrt(hidden_size)) 
# Plot weight columns as images (Visualizing hidden features)
fig, axes = plt.subplots(hidden_n, hidden_n, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    if i < rbm.h_units:
        img = rbm.W[:, i].reshape(img_size, img_size, channels)
        img = (img - img.min()) / (img.max() - img.min())  # Normalize for visualization
        ax.imshow(img, cmap='winter')
    ax.axis("off")
plt.suptitle("Weight Columns as Images (Hidden Features)")

plt.savefig("WeightColumnsasImages.png", format="png", dpi=300)  # Save as PNG
plt.savefig("WeightColumnsasImages.eps", format="eps")  # Save as PDF
plt.savefig("WeightColumnsasImages.pdf", format="pdf")  # Save as PDF
plt.savefig("WeightColumnsasImages.tiff", format="tiff", dpi=300)  # Save as TIFF

plt.show()




In [ ]:
"""
PLOT PROBABILITY DENSITY FUNCTION OF DATA
"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assume x_train is (400, 32, 32)
x1= data_flat.reshape(-1)  # Flatten all pixels into 1D

# Compute histogram (PDF estimation)
hist, bins = np.histogram(x1, bins=30, density=True)

# Compute bin centers
bin_centers = (bins[:-1] + bins[1:]) / 2

# Plot empirical PDF
plt.figure(figsize=(8, 5))
plt.plot(bin_centers, hist, linestyle='-', marker='o', label="Empirical PDF")
plt.xlabel("Pixel Intensity")
plt.ylabel("Density")
plt.title("Empirical PDF of Pixel Intensities")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
"""
Generate and plot new samples
"""


In [ ]:
# Generate and plot new face samples
generated_faces, generated_hidden_var= rbm.generate_samples(num_samples=100, steps=20)

fig, axes = plt.subplots(5, 5, figsize=(7, 7))
for i, ax in enumerate(axes.flat):
    img = generated_faces[i].reshape(img_size, img_size, channels)
    img =  (img - img.min()) / (img.max() - img.min())  # Normalize for visualization
    ax.imshow(img, cmap='winter')
    ax.axis("off")
    plt.suptitle("Generated Samples")  
plt.show()


In [ ]:
"""
Plot weight columns as images (Visualizing hidden features)
"""

In [ ]:

# Plot weight columns as images (Visualizing hidden features)
fig, axes = plt.subplots(5, 5, figsize=(7, 7))
for i, ax in enumerate(axes.flat):
    if i < rbm.h_units:
        img = rbm.W[:, i].reshape(img_size, img_size, channels)
        img = (img - img.min()) / (img.max() - img.min())  # Normalize for visualization
        ax.imshow(img, cmap='winter')
    ax.axis("off")
plt.suptitle("Weight Columns as Images (Hidden Features)")
plt.show()
